In [18]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [19]:
!pip install torchvision

In [20]:
import os
import torch
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

# Define dataset folder
dataset_path = "C:\\Users\\srini\\Downloads\\FruQ-DB\\FruQ-DB"  # Change this to your actual dataset folder

# Get all image files
image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Assign labels based on filename
labels = []
for img in image_files:
    if "rotten" in img.lower():  # If filename contains "rotten", label as 1
        labels.append(1)
    else:
        labels.append(0)  # Otherwise, label as 0 (fresh)

In [24]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch
from torchvision import transforms
import os

# Dummy data (Replace with your real file paths and labels)
image_files = ["C:\\Users\\srini\\Downloads\\FruQ-DB\\FruQ-DB\\Rotten\\Image91 (2).png", "C:\\Users\\srini\\Downloads\\FruQ-DB\\FruQ-DB\\Rotten\\Image93 (2).png"]
labels = [0, 1]

# Verify list lengths
assert len(image_files) == len(labels), "Mismatch in image and label count"

# Custom Dataset
class FruitDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label

# Transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create Dataset and Dataloader
fruit_dataset = FruitDataset(image_files, labels, transform=transform)

if len(fruit_dataset) == 0:
    print("❌ Dataset is empty. Check image paths or labels.")
else:
    train_loader = DataLoader(fruit_dataset, batch_size=32, shuffle=True)
    for images, labels in train_loader:
        print("✅ Batch shape:", images.shape)
        print("Labels:", labels)
        break


✅ Batch shape: torch.Size([2, 3, 128, 128])
Labels: tensor([0, 1])


In [25]:
class FruitDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        image = Image.open(img_path).convert("RGB")  # Open image
        if self.transform:
            image = self.transform(image)

        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create dataset and dataloader
fruit_dataset = FruitDataset(image_files, labels, transform=transform)
train_loader = DataLoader(fruit_dataset, batch_size=32, shuffle=True)

# Check dataset
for images, labels in train_loader:
    print("Batch shape:", images.shape)
    print("Labels:", labels)
    break

Batch shape: torch.Size([2, 3, 128, 128])
Labels: tensor([0, 1])


In [26]:
import torch.nn as nn
import torch.optim as optim

# Simple CNN model
class FruitCNN(nn.Module):
    def __init__(self):
        super(FruitCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: fresh & rotten
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(x.shape[0], -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model, loss, optimizer
model = FruitCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

Fruit Freshness code ,
RGB analysis ,
Training and testing.


In [27]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

# Data transforms for RGB images
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load dataset
data_dir = "C:\\Users\\srini\\Downloads\\FruQ-DB\\FruQ-DB"
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
class_names = dataset.classes

# Split into train/test
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


CNN model


In [28]:
class FruitCNN(nn.Module):
    def __init__(self, num_classes):
        super(FruitCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Use number of classes dynamically
num_classes = len(class_names)
model = FruitCNN(num_classes=num_classes).to(device)


Training loop with 10 epochs

In [29]:
import torch.optim as optim

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}")


Epoch [1/10], Loss: 0.4400
Epoch [2/10], Loss: 0.1460
Epoch [3/10], Loss: 0.0702
Epoch [4/10], Loss: 0.0578
Epoch [5/10], Loss: 0.0487
Epoch [6/10], Loss: 0.0376
Epoch [7/10], Loss: 0.0508
Epoch [8/10], Loss: 0.0450
Epoch [9/10], Loss: 0.0263
Epoch [10/10], Loss: 0.0223


In [30]:
print(class_names)
print("Number of classes:", len(class_names))


['Fresh', 'Mild', 'Rotten']
Number of classes: 3


Sample image prediction


In [34]:
from PIL import Image
import torch.nn.functional as F

def predict_image(image_path, model, transform, class_names):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dim

    with torch.no_grad():
        output = model(image)
        probs = F.softmax(output, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()
        confidence = torch.max(probs).item()

    return class_names[predicted_class], confidence

# Example usage
img_path = "C:\\Users\\srini\\Downloads\\FruQ-DB\\FruQ-DB\\Rotten\\Image93 (2).png"
label, confidence = predict_image(img_path, model, transform, class_names)
print(f"✅ Predicted: {label} ({confidence*100:.2f}% confidence)")


✅ Predicted: Rotten (99.22% confidence)


Sample Rotten Image Prediction


In [17]:
from PIL import Image
import torch.nn.functional as F

def predict_image(image_path, model, transform, class_names):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)  # Add batch dim

    with torch.no_grad():
        output = model(image)
        probs = F.softmax(output, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()
        confidence = torch.max(probs).item()

    return class_names[predicted_class], confidence

# Example usage
img_path = "C:\\Users\\srini\\Downloads\\image11.png"
label, confidence = predict_image(img_path, model, transform, class_names)
print(f"✅ Predicted: {label} ({confidence*100:.2f}% confidence)")

✅ Predicted: Rotten (100.00% confidence)


In [36]:
torch.save(model.state_dict(), "fruit_model.pth")


Model Summary


In [37]:
model.load_state_dict(torch.load("fruit_model.pth", map_location=device))
model.eval()


FruitCNN(
  (model): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Flatten(start_dim=1, end_dim=-1)
    (10): Linear(in_features=32768, out_features=256, bias=True)
    (11): ReLU()
    (12): Dropout(p=0.3, inplace=False)
    (13): Linear(in_features=256, out_features=3, bias=True)
  )
)

In [38]:
pip install torchinfo


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
print(os.getcwd())


c:\Users\srini\AppData\Local\Microsoft\Windows\INetCache\IE\W401EFVR


In [ ]:
import os

# Create 'models' directory if it doesn't exist
os.makedirs("models", exist_ok=True)

# Now save the model
torch.save(model.state_dict(), "models/fruit_model.pth")


In [ ]:
print(os.path.abspath("models/fruit_model.pth"))


c:\Users\srini\AppData\Local\Microsoft\Windows\INetCache\IE\W401EFVR\models\fruit_model.pth


In [ ]:
import zipfile
print(zipfile.is_zipfile("models/fruit_model.pth"))



True


In [6]:
pip install gradio


Note: you may need to restart the kernel to use updated packages.


In [14]:
import gradio as gr

def predict_by_filename(image):
    # Get filename (Gradio gives PIL Image with .filename attribute)
    filename = getattr(image, "filename", "")
    if not filename:
        return "Unknown"

    name = filename.lower()
    if "fresh" in name:
        return "fresh"
    elif "rotten" in name:
        return "rotten"
    else:
        return "Unknown (filename does not indicate freshness)"

iface = gr.Interface(
    fn=predict_by_filename,
    inputs=gr.Image(type="pil"),
    outputs="label",
    title="Fruit Freshness Predictor ",
    description="Predicts freshness based on image filename."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [1]:
import gradio as gr


c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import torch.nn as nn

class FruitCNN(nn.Module):
    def __init__(self, num_classes):
        super(FruitCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 32 * 32, 128)  # Adjust if input size changes
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [8]:
import gradio as gr

label_map_rev = {0: "fresh", 1: "rotten"}

def predict(image):
    image = transform(image).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(image)
        probs = torch.softmax(output, dim=1)
        confidence, predicted = torch.max(probs, 1)
        return {label_map_rev[0]: float(probs[0][0]), label_map_rev[1]: float(probs[0][1])}

iface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=2),
    title="Fruit Freshness Detector",
    description="Upload an image of a fruit to predict if it's fresh or rotten."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^

In [17]:
import gradio as gr
from PIL import Image
import os

# Path to dataset folder
dataset_path = "C:\\Users\\srini\\Downloads\\FruQ-DB"

# List image files from dataset
image_files = [f for f in os.listdir(dataset_path) if f.endswith((".jpg", ".png", ".jpeg"))]
image_paths = [os.path.join(dataset_path, f) for f in image_files]

def predict_from_path(image_path):
    # Rule-based dummy logic (filename-based prediction)
    name = image_path.lower()
    if "fresh" in name:
        label = "fresh"
    elif "rotten" in name:
        label = "rotten"
    else:
        label = "unknown"

    image = Image.open(image_path)
    return image, label

iface = gr.Interface(
    fn=predict_from_path,
    inputs=gr.Dropdown(choices=image_paths, label="Select Fruit Image from Dataset"),
    outputs=[gr.Image(type="pil"), gr.Label()],
    title="Fruit Freshness Predictor (from Dataset)",
    description="Choose an image from the dataset to predict freshness (no model used)."
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^

Created dataset file at: .gradio\flagged\dataset1.csv


Traceback (most recent call last):
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<5 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<11 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
    )
    ^
  File "c:\Users\srini\Desktop\food-freshness-backend\.venv\Lib\site-packages\gradio\blocks.py", line 1663, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^

In [19]:
import gradio as gr
from PIL import Image

def predict_by_filename(file):
    if file is None:
        return "No file uploaded"

    filename = file.name.lower()

    if "fresh" in filename:
        label = "fresh"
    elif "rotten" in filename or "notfresh" in filename:
        label = "rotten"
    else:
        label = "Unknown"

    # For visualization
    image = Image.open(file.name)
    return label, image

iface = gr.Interface(
    fn=predict_by_filename,
    inputs=gr.File(label="Upload a fruit image file"),
    outputs=["label", "image"],
    title="🍌 Fruit Freshness (Filename-Based Demo)",
    description="This demo guesses freshness based on the uploaded image's filename."
)

iface.launch()


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [21]:
import gradio as gr
from PIL import Image
import os

# Path to dataset folder
dataset_path = "C:\\Users\srini\\Desktop\\fruit_dataset"

# List image files from dataset
image_files = [f for f in os.listdir(dataset_path) if f.endswith((".jpg", ".png", ".jpeg"))]
image_paths = [os.path.join(dataset_path, f) for f in image_files]

def predict_from_path(image_path):
    # Rule-based dummy logic (filename-based prediction)
    name = image_path.lower()
    if "fresh" in name:
        label = "fresh"
    elif "rotten" in name:
        label = "rotten"
    else:
        label = "unknown"

    image = Image.open(image_path)
    return image, label

iface = gr.Interface(
    fn=predict_from_path,
    inputs=gr.Dropdown(choices=image_paths, label="Select Fruit Image from Dataset"),
    outputs=[gr.Image(type="pil"), gr.Label()],
    title="Fruit Freshness Predictor (from Dataset)",
    description="Choose an image from the dataset to predict freshness (no model used)."
)

iface.launch()

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:6: SyntaxWarning: invalid escape sequence '\s'
C:\Users\srini\AppData\Local\Temp\ipykernel_25320\1505219969.py:6: SyntaxWarning: invalid escape sequence '\s'
  dataset_path = "C:\\Users\srini\\Desktop\\fruit_dataset"


* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
